In [1]:
import shutil
import pandas as pd
import numpy as np
import SimpleITK as sitk
import os
from tqdm.notebook import tqdm
import cv2

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

from trainer.utils import split_mask, open_ct, open_mask
from trainer.loader import TailDs
from trainer.Trainer import Trainer
from nonlocalunet.models import NonLocalUnet, NonLocalUnetBuilder
from torchmed.images import to_numpy

import seaborn as sns

In [7]:
df = pd.read_csv('data/processed/df.csv')
train_df = df.loc[df.volume>19].reset_index(drop=True) 
val_df = df.loc[df.volume<20].reset_index(drop=True) 
test_indexes = list(range(0,2))

shape = [64, 128, 128]
train_batch_size = 48
val_batch_size = 48
split = False

train_ds = TailDs(train_df, split=split)
train_dl = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True, num_workers=8)

val_ds = TailDs(val_df, split=split)
val_dl = DataLoader(val_ds, batch_size=val_batch_size, shuffle=True, num_workers=8)

In [18]:
device = 'cuda:0'
depth = 3
init_filters = 32
lr = 0.0005
agg= False

model = NonLocalUnetBuilder(1, 7, depth=depth, init_filters=init_filters, agg=agg)
# model = torch.nn.DataParallel(model, device_ids=[1, 0, 2, 3], output_device=device)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = torch.nn.BCELoss()

name = f'{shape} b{train_batch_size} lr{lr} depth{depth} filt{init_filters} agg{int(agg)}'

In [4]:
trainer = Trainer(model, 7, shape, optimizer, scheduler, criterion, train_dl, val_dl, test_indexes, device, name)

In [ ]:
trainer.train(1000)

Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/101 [00:00<?, ?it/s]

Val:   0%|          | 0/19 [00:00<?, ?it/s]

Train:   0%|          | 0/101 [00:00<?, ?it/s]

Val:   0%|          | 0/19 [00:00<?, ?it/s]

...

Train:   0%|          | 0/101 [00:00<?, ?it/s]

Val:   0%|          | 0/19 [00:00<?, ?it/s]

Train:   0%|          | 0/101 [00:00<?, ?it/s]

Val:   0%|          | 0/19 [00:00<?, ?it/s]

Train:   0%|          | 0/101 [00:00<?, ?it/s]

In [26]:
loss = torch.nn.CrossEntropyLoss()

In [19]:
a, b = next(iter(train_dl))

In [20]:
preds = model(a.float().cuda())

In [25]:
preds.shape

torch.Size([48, 7, 16, 16, 16])

In [29]:
loss(preds, b.cuda().long())

tensor(2.0674, device='cuda:0', grad_fn=<NllLoss2DBackward>)